### Capstone Project Introduction  - Analysing the optimal neighbourhood to open a new restaurant in the Manhattan Borough of New York using Foursquare and New York Location Data 

For a large scale restaurant chain or business the location of a new restaurent can be very important, where a restuarent is located may determine how many people can access and visit it and therefore, the revenue a restaurant can generate. The number of competing restaurants in an area may also be a determining factor of how succesful a restaurant will be. My project intends to provide important information to stakeholders who may be looking to open a new restaurant in the Manhattan Borough of New York City. 

Using foursquare location data and New York City postcode data I will investigating the ideal neighbourhood to open a new restaurant in Manhattan. To assess this problem I will be using segmentation and clustering techniques to Identify help which neighbourhood would be a good location for a new restaurant. I will be assessing the frequency of restauarants in each neighbourhood in a given radius and I aim to find a neighbourhood which has a lower frequency of restuarent venus and therefore, could be an excellent location for a new restaurent. 